<a href="https://colab.research.google.com/github/sogmgm/Algorithm_practice/blob/main/nb/Qwen3_(4B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

Goal: To convert `Qwen3-4B-Base` into a reasoning model via GRPO by using OpenR1's Math dataset.

We first pre fine-tune the model to make GRPO skip trying to match formatting - this speeds GRPO up.

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-19 08:28:04 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-19 08:28:04 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.6.2: Fast Qwen3 patching. Transformers: 4.52.4. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 69.34%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048

tokenizer_config.json:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.6.2 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### GRPO chat template
Since we're using a base model, we should set a chat template. You can make your own chat template as well!
1. DeepSeek uses `<think>` and `</think>`, but this is **not** necessary - you can customize it however you like!
2. A `system_prompt` is recommended to at least guide the model's responses.

In [4]:
reasoning_start = "<start_working_out>" # Acts as <think>
reasoning_end   = "<end_working_out>"   # Acts as </think>
solution_start  = "<SOLUTION>"
solution_end    = "</SOLUTION>"

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""
system_prompt

'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>'

We create a simple chat template below. Notice `add_generation_prompt` includes prepending `<start_working_out>` to guide the model to start its reasoning process.

In [5]:
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

# Replace with out specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

Let's see how our chat template behaves on an example:

In [6]:
tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"{reasoning_start}I think it's 2.{reasoning_end}{solution_start}2{solution_end}"},
    {"role" : "user", "content" : "What is 2+2?"},
], tokenize = False, add_generation_prompt = True)

"You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|endoftext|>What is 1+1?<start_working_out>I think it's 2.<end_working_out><SOLUTION>2</SOLUTION><|endoftext|>What is 2+2?<start_working_out>"

In [7]:
# 토크나이저의 모든 특수토큰 확인
print("=== 기본 특수토큰들 ===")
print(f"eos_token: {tokenizer.eos_token}")
print(f"bos_token: {tokenizer.bos_token}")
print(f"pad_token: {tokenizer.pad_token}")
print(f"unk_token: {tokenizer.unk_token}")
print(f"sep_token: {tokenizer.sep_token}")
print(f"cls_token: {tokenizer.cls_token}")
print(f"mask_token: {tokenizer.mask_token}")

# 특수토큰 딕셔너리로 확인
print("\n=== special_tokens_map ===")
print(tokenizer.special_tokens_map)

# 모든 특수토큰 ID 확인
print("\n=== 특수토큰 ID들 ===")
print(f"eos_token_id: {tokenizer.eos_token_id}")
print(f"bos_token_id: {tokenizer.bos_token_id}")
print(f"pad_token_id: {tokenizer.pad_token_id}")
print(f"unk_token_id: {tokenizer.unk_token_id}")

=== 기본 특수토큰들 ===
eos_token: <|endoftext|>
bos_token: None
pad_token: <|vision_pad|>
unk_token: None
sep_token: None
cls_token: None
mask_token: None

=== special_tokens_map ===
{'eos_token': '<|endoftext|>', 'pad_token': '<|vision_pad|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}

=== 특수토큰 ID들 ===
eos_token_id: 151643
bos_token_id: None
pad_token_id: 151654
unk_token_id: None


In [8]:

# 현재 설정된 chat_template 확인
print("=== 현재 chat_template ===")
print(tokenizer.chat_template)

# 템플릿에서 사용되는 변수들 확인
import re
template_vars = re.findall(r'\{\{\s*(\w+)\s*\}\}', tokenizer.chat_template)
print(f"\n=== 템플릿에서 사용되는 변수들 ===")
print(set(template_vars))

=== 현재 chat_template ===
{% if messages[0]['role'] == 'system' %}{{ messages[0]['content'] + eos_token }}{% set loop_messages = messages[1:] %}{% else %}{{ 'You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION>' + eos_token }}{% set loop_messages = messages %}{% endif %}{% for message in loop_messages %}{% if message['role'] == 'user' %}{{ message['content'] }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<start_working_out>' }}{% endif %}

=== 템플릿에서 사용되는 변수들 ===
set()


##  비교테스트(base)

In [11]:
from unsloth import FastLanguageModel
import torch
from transformers import TextStreamer


# 2. 프롬프트 구성
system_prompt = "You are a helpful and accurate assistant."
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

# 3. 추론 실행
streamer = TextStreamer(tokenizer, skip_prompt=False)

_ = model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    temperature=0,
    max_new_tokens=1024,
    streamer=streamer,
)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


You are a helpful and accurate assistant.<|endoftext|>What is the sqrt of 101?<start_working_out> The square root of 101 is approximately 10.05.<|endoftext|>


In [12]:
from unsloth import FastLanguageModel
import torch
from transformers import TextStreamer


# 2. 프롬프트 구성
system_prompt = "You are a helpful and accurate assistant."
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "Find the positive real solution to the equation    cos x = xcorrect to five decimal places."},
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

# 3. 추론 실행
streamer = TextStreamer(tokenizer, skip_prompt=False)

_ = model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    temperature=0,
    max_new_tokens=1024,
    streamer=streamer,
)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


You are a helpful and accurate assistant.<|endoftext|>Find the positive real solution to the equation    cos x = xcorrect to five decimal places.<start_working_out> We can use the Newton-Raphson method to find the positive real solution to the equation cos(x) = x.

Let f(x) = cos(x) - x. We want to find the root of this function, i.e., the value of x for which f(x) = 0.

The Newton-Raphson method is an iterative method that uses the following formula to find the root:

x_n+1 = x_n - f(x_n) / f'(x_n)

where x_n is the current estimate of the root, and x_n+1 is the next estimate.

First, we need to find the derivative of f(x):

f'(x) = -sin(x) - 1

Now, we can start with an initial guess for x, say x_0 = 1, and iterate using the Newton-Raphson formula until we reach the desired accuracy.

x_1 = x_0 - f(x_0) / f'(x_0)
x_1 = 1 - (cos(1) - 1) / (-sin(1) - 1)
x_1 ≈ 0.75036

x_2 = x_1 - f(x_1) / f'(x_1)
x_2 = 0.75036 - (cos(0.75036) - 0.75036) / (-sin(0.75036) - 1)
x_2 ≈ 0.73911

x_3 = x_2 - 

In [13]:
from unsloth import FastLanguageModel
import torch
from transformers import TextStreamer


# 2. 프롬프트 구성
system_prompt = "You are a helpful and accurate assistant."
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "다음 방정식의 양의 실근을 소수점 다섯째 자리까지 구하시오.    x · e^x = 2"},
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

# 3. 추론 실행
streamer = TextStreamer(tokenizer, skip_prompt=False)

_ = model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    temperature=0,
    max_new_tokens=1024,
    streamer=streamer,
)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


You are a helpful and accurate assistant.<|endoftext|>다음 방정식의 양의 실근을 소수점 다섯째 자리까지 구하시오.    x · e^x = 2<start_working_out> To solve the equation \( x \cdot e^x = 2 \) for the positive real root, we can use numerical methods such as the Newton-Raphson method. Here's a step-by-step solution:

### Step 1: Define the function and its derivative
Let \( f(x) = x \cdot e^x - 2 \). We need to find the root of this function, i.e., the value of \( x \) such that \( f(x) = 0 \).

The derivative of \( f(x) \) is:
\[
f'(x) = e^x + x \cdot e^x = e^x (1 + x)
\]

### Step 2: Choose an initial guess
We can start with an initial guess \( x_0 \). Since \( x \cdot e^x = 2 \), and \( e^x \) grows rapidly, we can estimate that \( x \) is around 0.5 to 1. Let's choose \( x_0 = 0.5 \).

### Step 3: Apply the Newton-Raphson method
The Newton-Raphson formula is:
\[
x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}
\]

Let's perform the iterations:

#### Iteration 1:
\[
f(0.5) = 0.5 \cdot e^{0.5} - 2 \approx 0.5 \cdot 1.648

### Pre fine-tuning for formatting
We now use a subset of NVIDIA's [Open Math Reasoning dataset](https://huggingface.co/datasets/nvidia/OpenMathReasoning) which was filtered to only include high quality DeepSeek R1 traces.

We'll only filter ~59 or so examples to first "prime" / pre fine-tune the model to understand our custom GRPO formatting.

In [9]:
from datasets import load_dataset
import pandas as pd
import numpy as np

dataset = load_dataset("unsloth/OpenMathReasoning-mini", split = "cot")
dataset = dataset.to_pandas()[
    ["expected_answer", "problem", "generated_solution"]
]

# Try converting to number - if not, replace with NaN
is_number = pd.to_numeric(pd.Series(dataset["expected_answer"]), errors = "coerce").notnull()
# Select only numbers
dataset = dataset.iloc[np.where(is_number)[0]]

dataset

README.md:   0%|          | 0.00/603 [00:00<?, ?B/s]

data/cot-00000-of-00001.parquet:   0%|          | 0.00/106M [00:00<?, ?B/s]

Generating cot split:   0%|          | 0/19252 [00:00<?, ? examples/s]

KeyboardInterrupt: 

We have to format the dataset to follow our GRPO style formatting:

In [ ]:
def format_dataset(x):
    expected_answer = x["expected_answer"]
    problem = x["problem"]

    # Remove generated <think> and </think>
    thoughts = x["generated_solution"]
    thoughts = thoughts.replace("<think>", "").replace("</think>", "")

    # Strip newlines on left and right
    thoughts = thoughts.strip()
    # Add our custom formatting
    final_prompt = \
        reasoning_start + thoughts + reasoning_end + \
        solution_start + expected_answer + solution_end
    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : problem},
        {"role" : "assistant", "content" : final_prompt},
    ]

dataset["Messages"] = dataset.apply(format_dataset, axis = 1)

Check to see if it worked:

In [ ]:
tokenizer.apply_chat_template(dataset["Messages"][0], tokenize = False)

Let's truncate the pre fine-tuning dataset to `max_seq_length/2` since we don't want too long reasoning traces.

Note this might take 2 minutes!

In [ ]:
dataset["N"] = dataset["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))

dataset = dataset.loc[dataset["N"] <= max_seq_length/2].copy()
dataset.shape

We then tokenize the messages and convert it to a Hugging Face compatible dataset format:

In [ ]:
from datasets import Dataset

dataset["text"] = tokenizer.apply_chat_template(dataset["Messages"].values.tolist(), tokenize = False)
dataset = Dataset.from_pandas(dataset)
dataset

Let's now pre fine-tune the model so it follows our custom GRPO formatting!

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
trainer.train()

Let's check if the model has learnt to follow the custom format:

## 비교테스트 (only sft)

In [ ]:

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},  # ← 여기만 바꾸면 됨!
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
from transformers import TextStreamer

_ = model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    temperature=0,
    max_new_tokens=1024,
    streamer=TextStreamer(tokenizer, skip_prompt=False),
)


In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "Find the positive real solution to the equation    cos x = xcorrect to five decimal places."},
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
from transformers import TextStreamer

_ = model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    temperature=0,
    max_new_tokens=1024,
    streamer=TextStreamer(tokenizer, skip_prompt=False),
)


Yes it did follow the formatting! Great! Let's remove some items before the GRPO step

In [ ]:
del dataset
torch.cuda.empty_cache()
import gc
gc.collect()

### Data Prep
<a name="Data"></a>

We're using Hugging Face's [Open R1 Math dataset](https://huggingface.co/datasets/open-r1/DAPO-Math-17k-Processed). You can also utilize OpenAI's famous [GSM8K dataset](https://huggingface.co/datasets/openai/gsm8k)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("open-r1/DAPO-Math-17k-Processed", "en", split = "train")
dataset

Let's look at the first row:

In [ ]:
dataset[0]["prompt"]

In [ ]:
dataset[0]["solution"]

In GSM8K, ee notice all answers like about have a ####, so we extract it. But for the Open R1 dataset, we can skip the below.

In [ ]:
def extract_hash_answer(text):
    # if "####" not in text: return None
    # return text.split("####")[1].strip()
    return text
extract_hash_answer(dataset[0]["solution"])

Let's map the dataset! and see the first row:

In [ ]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["prompt"]},
    ],
    "answer": extract_hash_answer(x["solution"]),
})
dataset[0]

We create a regex format to match the reasoning sections and answers:

In [ ]:
import re

# Add optional EOS token matching
solution_end_regex = r"</SOLUTION>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

match_format = re.compile(
    rf"{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end_regex}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)
match_format

We verify it works:

In [ ]:
match_format.findall(
    "Let me think!<end_working_out>"\
    f"<SOLUTION>\n2\n</SOLUTION>",
)

In [ ]:
match_format.findall(
    "<start_working_out>Let me think!<end_working_out>"\
    f"<SOLUTION>  2  </SOLUTION>\n\n",
)

We now want to create a reward function to match the format exactly - we reward it with 3 points if it succeeds:

In [ ]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

If it fails, we want to reward the model if it at least follows the format partially, by counting each symbol:

In [ ]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!

        # No need to reward <start_working_out> since we always prepend it!
        # score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
        score += 0.5 if response.count(solution_start)  == 1 else -1.0
        score += 0.5 if response.count(solution_end)    == 1 else -1.0
        scores.append(score)
    return scores

Finally, we want to extract the generated answer, and reward or penalize it! We also reward it based on how close the answer is to the true one via ratios:

In [ ]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(-2.0)
            continue
        # Correct answer gets 5 points!
        if guess == true_answer:
            score += 5.0
        # Match if spaces are seen, but less reward
        elif guess.strip() == true_answer.strip():
            score += 3.5
        else:
            # We also reward it if the answer is close via ratios!
            # Ie if the answer is within some range, reward it!
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 2.0
                elif ratio >= 0.8 and ratio <= 1.2: score += 1.5
                else: score -= 2.5 # Penalize wrong answers
            except:
                score -= 4.5 # Penalize
        scores.append(score)
    return scores

Also sometimes it might not be 1 number as the answer, but like a sentence for example "The solution is $20" -> we extract 20.

We also remove possible commas for example as in 123,456

In [ ]:
match_numbers = re.compile(
    solution_start + r".*?[\s]{0,}([-]?[\d\.\,]{1,})",
    flags = re.MULTILINE | re.DOTALL
)
print(match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  123,456  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  -0.234  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>17</SOLUTION>"))

We now prepare our main function which will print out the generated responses and the true answer, along with another reward function which converts text to float via `float` and sees if it's the same.

In [ ]:
global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    # Print only every few steps
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-2.5)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            # Remove commas like in 123,456
            guess       = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
            continue
    return scores

Get the top 90% prompt length so we don't accidentally truncate them!

Ie we'll remove the top 10% long prompts.

In [ ]:
tokenized = dataset.map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["prompt"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
print(tokenizer.decode(tokenized[0]["tokens"]))
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length
dataset = dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [ ]:
max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
# For optional training + evaluation
# new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model, # sft된 모델
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args = training_args,
    train_dataset = dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,709 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 66,060,288/4,088,528,384 (1.62% trained)


********************Question:
Compute the number of positive integers that divide at least two of the integers in the set $\{1^1,2^2,3^3,4^4,5^5,6^6,7^7,8^8,9^9,10^{10}\}$. 
Answer:
22 
Response:
Okay, so I need to find the number of positive integers that divide at least two of the numbers in the set {1^1, 2^2, 3^3, ..., 10^10}. Hmm, let's start by understanding what the problem is asking. We have a set where each element is a number raised to its own power, from 1 to 10. The goal is to find how many positive integers are divisors of at least two of these numbers. 

First, I think I should list out the elements of the set to see what they are. Let me compute each one:

1. 1^1 = 1
2. 2^2 = 4
3. 3^3 = 27
4. 4^4 = 256
5. 5^5 = 3125
6. 6^6 = 46656
7. 7^7 = 823543
8. 8^8 = 16777216
9. 9^9 = 387420489
10. 10^10 = 10000000000

Wait, those are all the elements. Now, I need to find all positive integers that divide at least two of these numbers. The key here is that a number can divide multipl

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std,rewards / check_numbers / mean,rewards / check_numbers / std
1,0.006000,-5.500000,4.000000,1841.750000,1829.000000,1846.000000,0.750000,1829.000000,1829.000000,1829.000000,0.151143,0.750000,1.500000,-1.875000,2.250000,-2.125000,0.250000,-2.250000,0.500000
2,0.007400,-1.500000,4.000000,1517.000000,1307.000000,1846.000000,0.250000,1407.333374,1307.000000,1472.000000,0.185790,2.250000,1.500000,0.375000,2.250000,-2.375000,0.250000,-1.750000,0.500000
3,0.006900,-7.500000,0.000000,1846.000000,1846.000000,1846.000000,1.000000,0.000000,0.000000,0.000000,0.171631,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-2.500000,0.000000
4,0.005700,-7.500000,0.000000,1846.000000,1846.000000,1846.000000,1.000000,0.000000,0.000000,0.000000,0.143156,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-2.500000,0.000000
5,0.007700,13.000000,0.000000,1059.000000,611.000000,1308.000000,0.000000,1059.000000,611.000000,1308.000000,0.192055,3.000000,0.000000,1.500000,0.000000,5.000000,0.000000,3.500000,0.000000
6,0.004400,-7.500000,0.000000,1846.000000,1846.000000,1846.000000,1.000000,0.000000,0.000000,0.000000,0.111180,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-2.500000,0.000000
7,0.006600,-6.125000,2.750000,1846.000000,1846.000000,1846.000000,1.000000,0.000000,0.000000,0.000000,0.165968,0.000000,0.000000,-2.250000,1.500000,-2.000000,0.000000,-1.875000,1.250000
8,0.005400,-7.500000,0.000000,1846.000000,1846.000000,1846.000000,1.000000,0.000000,0.000000,0.000000,0.134025,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-2.500000,0.000000
9,0.005200,-5.500000,4.000000,1787.250000,1611.000000,1846.000000,0.750000,1611.000000,1611.000000,1611.000000,0.130707,0.750000,1.500000,-1.875000,2.250000,-2.125000,0.250000,-2.250000,0.500000
10,0.004800,-2.375000,10.250000,1827.250000,1771.000000,1846.000000,0.750000,1771.000000,1771.000000,1771.000000,0.118911,0.750000,1.500000,-1.875000,2.250000,-0.250000,3.500000,-1.000000,3.000000


********************Question:
The sum of $\lfloor x \rfloor$ for all real numbers $x$ satisfying the equation $16 + 15x + 15x^2 = \lfloor x \rfloor^3$ is: 
Answer:
33 
Response:
Okay, so I need to solve this equation: 16 + 15x + 15x² = ⌊x⌋³. The goal is to find all real numbers x that satisfy this equation and then sum up their floor values, ⌊x⌋. Hmm, let's break this down.

First, I remember that ⌊x⌋ is the greatest integer less than or equal to x. So, ⌊x⌋ is an integer, and the equation involves that integer cubed being equal to a quadratic expression in x. That might be a hint that x is close to an integer. Let me think about possible integer values for ⌊x⌋ and see if any of them work.

Let's denote k = ⌊x⌋, where k is an integer. Then the equation becomes 16 + 15x + 15x² = k³. Since k is the floor of x, we know that k ≤ x < k+1. So x is in the interval [k, k+1).

Now, substitute x with k + t, where t is a real number between 0 and 1 (since x is between k and k+1). Then the equation

TrainOutput(global_step=100, training_loss=0.005594823628198356, metrics={'train_runtime': 11085.4605, 'train_samples_per_second': 0.036, 'train_steps_per_second': 0.009, 'total_flos': 0.0, 'train_loss': 0.005594823628198356})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = "What is the sqrt of 101?"

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

' Round to the hundredths place - Brainly.com\nBreez\n01/10/2015\nMathematics\nMiddle School\nanswered • expert verified\nWhat is the sqrt of 101? Round to the hundredths place\nAdvertisement\nBreez is waiting for your help.\nAdd your answer and earn points.\nExpert-verified answer\n1\napologiabiology\nthe sqareroot of 100 is 10 so the sqaretooot of 101 is close to 10\n10 to the power of 2 is 100 so lets see the sqare root of 11 to see how close the sqareoot of 101 is\nsqrt(11) is 3.31548464 and this is close to 3.25 so lets use 3.25\n10+3.25=13.25\n13.25*13.25=175.5625\n175.5625>101\nso sqrt(101)=13.25-something beucase 13.25 times 13.25 is too big\nlets try more smaller numbers\n13.24\n13.24*13.24=175.3376 too big\n13.23\n13.23*13.23=175.1929 too big\n13.22\n13.22*13.22=174.7684 too big but close\nlets try between 13.22 and 13.23\ntry 13.225\n13.225*13.225=174.900625\ntoo big try 13.2225\n13.2225*13.2225=174.83200625\nclose but too big try 13.2205\n13.2205*13.2205=174.76728025 close 

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
# model.save_lora("grpo_saved_lora")

In [24]:
# from datetime import datetime

# # 오늘 날짜를 "YYYYMMDD" 형식으로 가져오기
# date_str = datetime.now().strftime("%Y%m%d")

# # 저장 경로 지정
# save_dir = f"/content/drive/MyDrive/LLM/grpo/grpo_saved_lora_{date_str}"

# # 저장
# model.save_lora(save_dir)


In [33]:
!rm -rf /content/drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls /content/grpo_saved_lora


adapter_config.json  adapter_model.safetensors	README.md


Verify LoRA is actually trained!

In [34]:
# date_str-1

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [35]:
from safetensors import safe_open

tensors = {}
# with safe_open(f"/content/drive/MyDrive/LLM/grpo/grpo_saved_lora_{date_str}/adapter_model.safetensors", framework = "pt") as f:
with safe_open(f"/content/drive/MyDrive/LLM/grpo/grpo_saved_lora_20250618/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

Now we load the LoRA and test:

In [36]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    # lora_request = model.load_lora("grpo_saved_lora"),
    # lora_request = model.load_lora(f"/content/drive/MyDrive/LLM/grpo/grpo_saved_lora_{date_str}"), # 금일 훈련한 모델일 경우
    lora_request = model.load_lora(f"/content/drive/MyDrive/LLM/grpo/grpo_saved_lora_20250618"), # 250618은 gpro 훈련된 버전임
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

"Okay, so I need to find the square root of 101. Hmm, let me think. I remember that the square root of a number is a value that, when multiplied by itself, gives the original number. So, sqrt(101) is a number x such that x^2 = 101. But wait, I'm not sure what the exact value of sqrt(101) is. Maybe I can estimate it?\n\nFirst, I know that 100 is a perfect square, specifically 10^2. So sqrt(100) = 10. That means sqrt(101) should be slightly more than 10 because 101 is just 1 more than 100. Let me see; maybe around 10.05 or something? But I need a more precise value. Hmm, maybe I can use the method of approximation or a calculator. But since this is a math problem, maybe there's a better way.\n\nWait, what if I try to express 101 as a perfect square plus a remainder? Let me think. 101 divided by 100 is 1 with a remainder of 1, so 101 = 10^2 + 1. So sqrt(101) = sqrt(100 + 1). Maybe Taylor series can help here? But that might be too complicated. Or maybe I can use the fact that sqrt(a^2 + b

## 비교 테스트 (SFT -> GRPO)

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora(f"/content/drive/MyDrive/LLM/grpo/grpo_saved_lora_20250618"), # 250618은 gpro 훈련된 버전임
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

"Okay, let's find the square root of 101. Hmm, I remember that square roots of non-perfect squares are irrational numbers. So, 101 doesn't have an exact square root that's a whole number. But maybe I can approximate it. Let me think about some perfect squares around 101. The square root of 100 is 10, and the square root of 121 is 11. So, the square root of 101 has to be between 10 and 11. \n\nLet me try using the Babylonian method or an approximation. Starting with an initial guess, like 10. To refine it, divide 101 by 10 to get 10.1. Then take the average of 10 and 10.1, which is 10.05. Now, square 10.05: 10.05 * 10.05. Let me do that. 10 * 10 is 100, 10*0.05 is 0.5, and 0.05*10 is 0.5, and 0.05*0.05 is 0.0025. Adding those up; 100 + 0.5 + 0.5 + 0.0025 = 101.0025, which is close to 101. So, 10.05 is pretty good. \n\nAlternatively, using the formula for approximating square roots: sqrt(a) is approximately (x + y)/2 where x^2 < a < (x+1)^2 and y is the quotient when dividing a by x. For

In [40]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "Find the positive real solution to the equation    sin x = x² correct to five decimal places."},

]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 4048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora(f"/content/drive/MyDrive/LLM/grpo/grpo_saved_lora_20250618"), # 250618은 gpro 훈련된 버전임
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

"Okay, so I need to find the positive real solution to the equation sin(x) = x², accurate to five decimal places. Hmm, let me think. I remember that sometimes these types of equations can't be solved algebraically, so maybe I'll need to use numerical methods like the Newton-Raphson method. Let me recall how that works. The Newton-Raphson formula is x_{n+1} = x_n - f(x)/f'(x), and I can use it to get close to a root. But first, I need to define the function. Since the equation is sin(x) = x², I can rearrange it to f(x) = x² - sin(x) = 0. Then, the derivative f'(x) would be 2x - cos(x). Alright, so now I have to choose an initial guess. The problem asks for the positive solution. Let me think about where the solutions might be located. I know that sin(x) oscillates between -1 and 1, and x² is always non-negative. So, where cos x is zero, say at π/2, then sin(π/2) = 1, and π²/4 ≈ 2.4674, which is greater than 1. So, somewhere between 0 and π/2, there must be a solution. Let me try x = 0.5

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

In [ ]:
import os
os.listdir("/content/model")


FileNotFoundError: [Errno 2] No such file or directory: '/content/model'

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
